In [1]:
import pandas as pd
import configparser

In [2]:
config = configparser.RawConfigParser()
config.read('/Users/zachblumenfeld/devtools/aura-freight-demo.ini')
HOST = config['NEO4J']['HOST']
USERNAME = config['NEO4J']['USERNAME']
PASSWORD = config['NEO4J']['PASSWORD']

In [3]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=True)

In [4]:
# Clear last graph - All data and schema attributes
gds.run_cypher('MATCH(n) DETACH DELETE n')
gds.run_cypher('CALL apoc.schema.assert({},{})')

,label,key,keys,unique,action
0,FREIGHT_RECEPTION,shipmentId,[shipmentId],False,DROPPED
1,FREIGHT_DEPARTURE,shipmentId,[shipmentId],False,DROPPED
2,FREIGHT_TRANSPORT,shipmentId,[shipmentId],False,DROPPED
3,CONNECT,shipmentId,[shipmentId],False,DROPPED
4,FREIGHT_DELIVERY,shipmentId,[shipmentId],False,DROPPED
5,TRANSFER,shipmentId,[shipmentId],False,DROPPED
6,DeparturePoint,airportId,[airportId],True,DROPPED
7,ArrivalWarehouse,airportId,[airportId],True,DROPPED
8,Destination,airportId,[airportId],True,DROPPED
9,EntryPoint,airportId,[airportId],True,DROPPED


In [5]:
# get source data from: https://s-cube-network.eu/c2k-files/c2k_data_comma.csv
df = pd.read_csv('https://s-cube-network.eu/c2k-files/c2k_data_comma.csv', dtype=str)

In [6]:
aa  = [i for i in df.columns if 'place' in i]
aa.sort()

In [7]:
INBOUND_LEGS= ['i1','i2','i3']
SEGMENTS = [1,2,3]

def get_path_segment(row, leg, segment):
    f = row[f'{leg}_dep_{segment}_place']
    res = ''
    if f!='?':
        t = row[f'{leg}_rcf_{segment}_place']
        res = f'-{f}dep-{t}rcf'
    return res

def get_path_indexes(row):
    res = []
    if pd.notnull(row.legs):
        outbound_path = ''
        for s in SEGMENTS:
            outbound_path = outbound_path + get_path_segment(row, 'o', s)
        for i in range(1, int(row.legs) + 1):
            inbound_path = ''
            for s in SEGMENTS:
                inbound_path = inbound_path + get_path_segment(row, f'i{i}', s)
                res.append(inbound_path[1:] + outbound_path)
    return res

df['path_indexes'] = df.apply(get_path_indexes, axis=1)
df

,nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,...,o_dep_3_e,o_dep_3_place,o_rcf_3_p,o_rcf_3_e,o_rcf_3_place,o_dlv_p,o_dlv_e,o_hops,legs,path_indexes
0,0,5182,199,218,210,215,609,935,736,256,...,?,?,?,?,?,780,434,1,2,"[609dep-256rcf-256dep-411rcf, 609dep-256rcf-25..."
1,1,6523,844,584,90,297,700,1935,1415,431,...,?,?,?,?,?,3870,445,1,2,"[700dep-431rcf-431dep-256rcf, 700dep-431rcf-43..."
2,2,5878,4380,4119,90,280,456,905,547,700,...,?,?,?,?,?,550,1520,1,1,"[456dep-700rcf-700dep-349rcf, 456dep-700rcf-70..."
3,3,1275,759,169,240,777,173,340,577,349,...,?,?,?,?,?,3780,159,1,1,"[173dep-349rcf-671dep-700rcf, 173dep-349rcf-34..."
4,4,8117,1597,1485,150,241,411,585,612,128,...,?,?,?,?,?,4140,4797,2,1,"[411dep-128rcf-166dep-128rcf-128dep-411rcf, 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,3939,4225,122,19,240,278,815,830,761,174,...,?,?,?,?,?,1665,1300,2,3,"[815dep-174rcf-174dep-349rcf-349dep-293rcf, 81..."
3939,3940,14017,2028,413,270,1825,605,2610,2535,349,...,?,?,?,?,?,3780,807,2,1,"[605dep-349rcf-597dep-700rcf-700dep-113rcf, 60..."
3940,3941,4660,1356,178,240,1359,815,760,716,609,...,?,?,?,?,?,5100,4381,2,1,"[815dep-609rcf-609dep-815rcf-815dep-737rcf, 81..."
3941,3942,6472,2692,1856,90,867,700,1060,1049,113,...,?,?,?,?,?,3780,945,2,2,"[700dep-113rcf-113dep-700rcf-700dep-635rcf, 70..."


In [8]:
gds.run_cypher('CREATE CONSTRAINT airport_unique IF NOT EXISTS ON (n:Airport) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE CONSTRAINT airport_entry_unique IF NOT EXISTS ON (n:EntryPoint) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE INDEX check_in_shipment_id IF NOT EXISTS FOR ()-[r:FREIGHT_RECEPTION]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_checkpoint_unique IF NOT EXISTS ON (n:DepartureWarehouse) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE INDEX confirm_shipment_id IF NOT EXISTS FOR ()-[r:FREIGHT_DEPARTURE]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_unique IF NOT EXISTS ON (n:DeparturePoint) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE INDEX depart_shipment_id IF NOT EXISTS FOR ()-[r:FREIGHT_TRANSPORT]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_arrival_unique IF NOT EXISTS ON (n:ArrivalWarehouse) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE INDEX connect_shipment_id IF NOT EXISTS FOR ()-[r:CONNECT]-() ON (r.shipmentId)')

gds.run_cypher('CREATE INDEX deliver_shipment_id IF NOT EXISTS FOR ()-[r:FREIGHT_DELIVERY]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_destination_unique  IF NOT EXISTS ON (n:Destination) ASSERT n.airportId  IS UNIQUE')

gds.run_cypher('CREATE INDEX transfer_shipment_id IF NOT EXISTS FOR ()-[r:TRANSFER]-() ON (r.shipmentId)')



""


In [9]:
# Load Nodes
cols = df.columns.tolist()

airport_ids = set()
for col in cols:
    if 'place' in col:
        #print([i for i in df.loc[(df[col] != '?') & (df[col].notna()), col].tolist()])
        airport_ids.update([int(i) for i in df.loc[(df[col] != '?') & (df[col].notna()), col].unique().tolist()])

gds.run_cypher('''
    UNWIND $airportIds AS airportId
    MERGE(n0:Airport {airportId: airportId})
    MERGE(n1:EntryPoint {airportId: airportId})
    MERGE(n2:DepartureWarehouse{airportId: airportId})
    MERGE(n3:DeparturePoint {airportId: airportId})
    MERGE(n4:ArrivalWarehouse {airportId: airportId})
    MERGE(n5:Destination {airportId: airportId})
    RETURN count(n0), count(n1), count(n2), count(n3), count(n4), count(n5)
    ''', params={'airportIds':list(airport_ids)})

,count(n0),count(n1),count(n2),count(n3),count(n4),count(n5)
0,237,237,237,237,237,237


In [10]:
gds.run_cypher('''
    UNWIND $airportIds AS airportId
    MATCH(n0:Airport {airportId: airportId})
    MATCH(n1:EntryPoint {airportId: airportId})
    MATCH(n2:DepartureWarehouse {airportId: airportId})
    MATCH(n3:DeparturePoint {airportId: airportId})
    MATCH(n4:ArrivalWarehouse {airportId: airportId})
    MATCH(n5:Destination {airportId: airportId})

    MERGE(n0)<-[r1:LOCATED_AT]-(n1)
    MERGE(n0)<-[r2:LOCATED_AT]-(n2)
    MERGE(n0)<-[r3:LOCATED_AT]-(n3)
    MERGE(n0)<-[r4:LOCATED_AT]-(n4)
    MERGE(n0)<-[r5:LOCATED_AT]-(n5)
    RETURN count(r1), count(r2), count(r3), count(r4), count(r5)
    ''', params={'airportIds':list(airport_ids)})

,count(r1),count(r2),count(r3),count(r4),count(r5)
0,237,237,237,237,237


In [11]:
LEGS = [1,2,3]
SEGMENTS = [1,2,3]
GOINGS = ['i','o']
SERVICES = ['rcs', 'dlv']

In [12]:
# (n:EntryPoint)-[r:FREIGHT_RECEPTION]-(m:DepartureWarehouse) RCS inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_rcs_p', f'i{l}_rcs_e', f'i{l}_dep_1_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH relMap.nr AS shipmentId,
            relMap.i{l}_dep_1_place AS airportId,
            relMap.i{l}_legid AS legId,
            relMap.i{l}_rcs_e AS effectiveMinutes,
            relMap.i{l}_rcs_p AS plannedMinutes
        MATCH(n1:EntryPoint {{airportId: airportId}})
        MATCH(n2:DepartureWarehouse {{airportId: airportId}})
        MERGE(n1)-[r:FREIGHT_RECEPTION {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [13]:
# (n:EntryPoint)-[r:FREIGHT_RECEPTION]-(m:DepartureWarehouse) RCS outbound

sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', f'o_legid', f'o_rcs_p', f'o_rcs_e', f'o_dep_1_place']].astype(int).to_dict('records')
gds.run_cypher('''
    UNWIND $relMaps AS relMap
    WITH relMap.nr AS shipmentId,
        relMap.o_dep_1_place AS airportId,
        relMap.o_legid AS legId,
        relMap.o_rcs_e AS effectiveMinutes,
        relMap.o_rcs_p AS plannedMinutes
    MATCH(n1:EntryPoint {airportId: airportId})
    MATCH(n2:DepartureWarehouse {airportId: airportId})
    MERGE(n1)-[r:FREIGHT_RECEPTION {shipmentId: shipmentId, legId: legId, legNumber: -1}]->(n2)
    ON CREATE SET r.plannedMinutes=plannedMinutes,
        r.effectiveMinutes=effectiveMinutes
    RETURN count(r)
''', params={'relMaps':sub_dict})

,count(r)
0,3942


In [14]:
# (:DepartureWarehouse)-[r:FREIGHT_DEPARTURE]-(:DeparturePoint) DEP1 inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_dep_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
            ['nr', f'i{l}_legid', f'i{l}_dep_{s}_p', f'i{l}_dep_{s}_e', f'i{l}_dep_{s}_place']]\
            .astype(int).to_dict('records')
        res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH relMap.nr AS shipmentId,
                relMap.i{l}_dep_{s}_place AS airportId,
                relMap.i{l}_legid AS legId,
                relMap.i{l}_dep_{s}_e AS effectiveMinutes,
                relMap.i{l}_dep_{s}_p AS plannedMinutes
            MATCH(n1:DepartureWarehouse {{airportId: airportId}})
            MATCH(n2:DeparturePoint {{airportId: airportId}})
            MERGE(n1)-[r:FREIGHT_DEPARTURE {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=plannedMinutes,
                r.effectiveMinutes=effectiveMinutes
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [15]:
# (:DepartureWarehouse)-[r:FREIGHT_DEPARTURE]-(:DeparturePoint) DEP1 outbound
for s in SEGMENTS:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_dep_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_dep_{s}_p', f'o_dep_{s}_e', f'o_dep_{s}_place']]\
        .astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH relMap.nr AS shipmentId,
            relMap.o_dep_{s}_place AS airportId,
            relMap.o_legid AS legId,
            relMap.o_dep_{s}_e AS effectiveMinutes,
            relMap.o_dep_{s}_p AS plannedMinutes
        MATCH(n1:DepartureWarehouse {{airportId: airportId}})
        MATCH(n2:DeparturePoint {{airportId: airportId}})
        MERGE(n1)-[r:FREIGHT_DEPARTURE {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [16]:
# (:DeparturePoint)-[r:FREIGHT_TRANSPORT]-(:ArrivalWarehouse) inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_rcf_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
                          ['nr', f'i{l}_legid', f'i{l}_rcf_{s}_p', f'i{l}_rcf_{s}_e', f'i{l}_dep_{s}_place', f'i{l}_rcf_{s}_place']]\
            .astype(int).to_dict('records')
        res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH relMap.nr AS shipmentId,
                relMap.i{l}_dep_{s}_place AS fromAirportId,
                relMap.i{l}_rcf_{s}_place AS toAirportId,
                relMap.i{l}_legid AS legId,
                relMap.i{l}_rcf_{s}_e AS effectiveMinutes,
                relMap.i{l}_rcf_{s}_p AS plannedMinutes
            MATCH(n1:DeparturePoint {{airportId: fromAirportId}})
            MATCH(n2:ArrivalWarehouse {{airportId: toAirportId}})
            MERGE(n1)-[r:FREIGHT_TRANSPORT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=plannedMinutes,
                r.effectiveMinutes=effectiveMinutes
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [17]:
# (:DeparturePoint)-[r:FREIGHT_TRANSPORT]-(:ArrivalWarehouse) RCF outbound
for s in SEGMENTS:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_rcf_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_rcf_{s}_p', f'o_rcf_{s}_e',  f'o_dep_{s}_place', f'o_rcf_{s}_place']]\
        .astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH relMap.nr AS shipmentId,
            relMap.o_dep_{s}_place AS fromAirportId,
            relMap.o_rcf_{s}_place AS toAirportId,
            relMap.o_legid AS legId,
            relMap.o_rcf_{s}_e AS effectiveMinutes,
            relMap.o_rcf_{s}_p AS plannedMinutes
        MATCH(n1:DeparturePoint {{airportId: fromAirportId}})
        MATCH(n2:ArrivalWarehouse {{airportId: toAirportId}})
        MERGE(n1)-[r:FREIGHT_TRANSPORT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [18]:
# assign last inbound & outbound rcf place
def get_last_i1_rcf_place(row):
    res = row[f'i1_rcf_1_place']
    for s in [3,2]:
        if row[f'i1_rcf_{s}_place'] != '?':
            res = row[f'i1_rcf_{s}_place']
    return res
df['last_i1_rcf_place'] = df.apply(get_last_i1_rcf_place, axis =1)

def get_last_i2_rcf_place(row):
    res = row[f'i2_rcf_1_place']
    for s in [3,2]:
        if row[f'i2_rcf_{s}_place'] != '?':
            res = row[f'i2_rcf_{s}_place']
    return res
df['last_i2_rcf_place'] = df.apply(get_last_i2_rcf_place, axis =1)

def get_last_i3_rcf_place(row):
    res = row[f'i3_rcf_1_place']
    for s in [3,2]:
        if row[f'i3_rcf_{s}_place'] != '?':
            res = row[f'i3_rcf_{s}_place']
    return res
df['last_i3_rcf_place'] = df.apply(get_last_i3_rcf_place, axis =1)

def get_last_outbound_rcf_place(row):
    for s in [3,2,1]:
        if row[f'o_rcf_{s}_place'] != '?':
            return row[f'o_rcf_{s}_place']
    raise Exception("cannot find last rcf place")
df['last_o_rcf_place'] = df.apply(get_last_outbound_rcf_place, axis =1)

df

,nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,...,o_rcf_3_place,o_dlv_p,o_dlv_e,o_hops,legs,path_indexes,last_i1_rcf_place,last_i2_rcf_place,last_i3_rcf_place,last_o_rcf_place
0,0,5182,199,218,210,215,609,935,736,256,...,?,780,434,1,2,"[609dep-256rcf-256dep-411rcf, 609dep-256rcf-25...",256,256,?,411
1,1,6523,844,584,90,297,700,1935,1415,431,...,?,3870,445,1,2,"[700dep-431rcf-431dep-256rcf, 700dep-431rcf-43...",431,431,?,256
2,2,5878,4380,4119,90,280,456,905,547,700,...,?,550,1520,1,1,"[456dep-700rcf-700dep-349rcf, 456dep-700rcf-70...",700,?,?,349
3,3,1275,759,169,240,777,173,340,577,349,...,?,3780,159,1,1,"[173dep-349rcf-671dep-700rcf, 173dep-349rcf-34...",671,?,?,700
4,4,8117,1597,1485,150,241,411,585,612,128,...,?,4140,4797,2,1,"[411dep-128rcf-166dep-128rcf-128dep-411rcf, 41...",166,?,?,411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,3939,4225,122,19,240,278,815,830,761,174,...,?,1665,1300,2,3,"[815dep-174rcf-174dep-349rcf-349dep-293rcf, 81...",174,174,174,293
3939,3940,14017,2028,413,270,1825,605,2610,2535,349,...,?,3780,807,2,1,"[605dep-349rcf-597dep-700rcf-700dep-113rcf, 60...",597,?,?,113
3940,3941,4660,1356,178,240,1359,815,760,716,609,...,?,5100,4381,2,1,"[815dep-609rcf-609dep-815rcf-815dep-737rcf, 81...",609,?,?,737
3941,3942,6472,2692,1856,90,867,700,1060,1049,113,...,?,3780,945,2,2,"[700dep-113rcf-113dep-700rcf-700dep-635rcf, 70...",113,113,?,635


In [19]:
# (:ArrivalWarehouse)-[r:FREIGHT_DELIVERY]-(:Destination) DLV inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_dlv_p', f'i{l}_dlv_e', 'last_i1_rcf_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH relMap.nr AS shipmentId,
            relMap.last_i1_rcf_place AS airportId,
            relMap.i{l}_legid AS legId,
            relMap.i{l}_dlv_e AS effectiveMinutes,
            relMap.i{l}_dlv_p AS plannedMinutes
        MATCH(n1:ArrivalWarehouse {{airportId: airportId}})
        MATCH(n2:Destination {{airportId: airportId}})
        MERGE(n1)-[r:FREIGHT_DELIVERY {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [20]:
# (:ArrivalWarehouse)-[r:FREIGHT_DELIVERY]-(:Destination) DLV outbound
sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', 'o_legid', f'o_dlv_p', f'o_dlv_e', 'last_o_rcf_place']].astype(int).to_dict('records')
res = gds.run_cypher('''
    UNWIND $relMaps AS relMap
    WITH relMap.nr AS shipmentId,
        relMap.last_o_rcf_place AS airportId,
        relMap.o_legid AS legId,
        relMap.o_dlv_e AS effectiveMinutes,
        relMap.o_dlv_p AS plannedMinutes
    MATCH(n1:ArrivalWarehouse {airportId: airportId})
    MATCH(n2:Destination {airportId: airportId})
    MERGE(n1)-[r:FREIGHT_DELIVERY {shipmentId: shipmentId, legId: legId, legNumber: $legNumber}]->(n2)
    ON CREATE SET r.plannedMinutes=plannedMinutes,
        r.effectiveMinutes=effectiveMinutes
    RETURN count(r)
''', params={'relMaps':sub_dict, 'legNumber':-1})
print(res)

   count(r)
0      3942


In [21]:
# (:Destination)-[r:TRANSFER]-(:EntryPoint) inbound->outbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', 'last_i1_rcf_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH relMap.nr AS shipmentId,
            relMap.last_i1_rcf_place AS airportId,
            relMap.i{l}_legid AS legId
        MATCH(n1:Destination {{airportId: airportId}})
        MATCH(n2:EntryPoint {{airportId: airportId}})
        MERGE(n1)-[r:TRANSFER {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=0, r.effectiveMinutes=0
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [22]:
# (:ArrivalWarehouse)-[r:CONNECT]-(:DepartureWarehouse) inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in [2,3]:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_rcf_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
                          ['nr', f'i{l}_legid', f'i{l}_dep_{s}_place']].astype(int).to_dict('records')
        res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH relMap.nr AS shipmentId,
                relMap.i{l}_dep_{s}_place AS airportId,
                relMap.i{l}_legid AS legId
            MATCH(n1:ArrivalWarehouse {{airportId: airportId}})
            MATCH(n2:DepartureWarehouse {{airportId: airportId}})
            MERGE(n1)-[r:CONNECT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=0, r.effectiveMinutes=0
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l})
        print(res)

== LEG 1 ======================
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [23]:
# (:ArrivalWarehouse)-[r:CONNECT]-(:DepartureWarehouse) outbound

for s in [2,3]:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_rcf_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_dep_{s}_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH relMap.nr AS shipmentId,
                relMap.o_dep_{s}_place AS airportId,
                relMap.o_legid AS legId
            MATCH(n1:ArrivalWarehouse {{airportId: airportId}})
            MATCH(n2:DepartureWarehouse {{airportId: airportId}})
            MERGE(n1)-[r:CONNECT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=0, r.effectiveMinutes=0
            RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1})
    print(res)

-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26
